In [2]:
!pip install lpips

In [2]:
!pip install opencv-python

# FINAL CALCULATIONS
## SSIM AND PSNR

In [3]:
!pip install opencv-python

In [4]:
import os
import cv2
import numpy as np
from skimage.metrics import structural_similarity as ssim

In [5]:
def calculate_psnr(img1, img2):
    mse = np.mean((img1 - img2) ** 2)
    if mse == 0:
        return float('inf')
    max_pixel = 255.0
    psnr = 20 * np.log10(max_pixel / np.sqrt(mse))
    return psnr

def calculate_ssim(img1, img2):
    gray_img1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
    gray_img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)
    return ssim(gray_img1, gray_img2, full=True)[0]

'''
def calculate_ssim(img1, img2):
    gray_img1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
    gray_img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)
    (_, ssim_value) = ssim(gray_img1, gray_img2, full=True)
    return ssim_value
'''

'\ndef calculate_ssim(img1, img2):\n    gray_img1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)\n    gray_img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)\n    (_, ssim_value) = ssim(gray_img1, gray_img2, full=True)\n    return ssim_value\n'

# ERGAS 

In [6]:
!pip install sewar

  Preparing metadata (setup.py) ... done
  Created wheel for sewar: filename=sewar-0.4.6-py3-none-any.whl size=11420 sha256=908ada6efe1a4084e4370650b51d3c56cd67a0b6af51b390a51a533bbe1cdb67
  Stored in directory: /root/.cache/pip/wheels/3f/af/02/9c6556ba287b62a945d737def09b8b8c35c9e1d82b9dfae84c
Successfully built sewar


In [7]:
import numpy as np
import torch
import torch.nn.functional as F
from skimage.metrics import mean_squared_error, structural_similarity
from scipy.ndimage import gaussian_filter
from scipy.linalg import norm
from math import sqrt, acos, pi
from skimage import img_as_float
from skimage.util import img_as_ubyte
from sewar.full_ref import ergas as sewar_ergas
from sewar.full_ref import sam as sewar_sam
from sewar.full_ref import vifp as sewar_vifp
from skimage.transform import resize

In [8]:
def calculate_ergas(true_img, degraded_img):
    true_img = img_as_float(true_img)
    degraded_img = img_as_float(degraded_img)

    degraded_img = resize(degraded_img, true_img.shape, anti_aliasing=True)
    # Global Dimensionless Relative Error of Synthesis (ERGAS)
    ergas_val = sewar_ergas(true_img, degraded_img)
    return ergas_val

## Baseline with microsoft 
not available since we dont have the original degraded image

## degradation baseline with Pix2Pix 

In [10]:
def main():
    real_dir = '/kaggle/input/baseline'
    fake_dir = '/kaggle/input/degraded-ours'

    real_files = os.listdir(real_dir)
    fake_files = os.listdir(fake_dir)
    
    real_files = sorted(os.listdir(real_dir))
    fake_files = sorted(os.listdir(fake_dir))
    #print(fake_files)
    #print(real_files)
    psnr_values = []
    ssim_values = []
    ergas_values = []

    for real_file, fake_file in zip(real_files, fake_files):
        if real_file.endswith('.jpeg') and fake_file.endswith('.jpeg'):
            real_img = cv2.imread(os.path.join(real_dir, real_file))
            fake_img = cv2.imread(os.path.join(fake_dir, fake_file))

            if real_img.shape != fake_img.shape:
                fake_img = cv2.resize(fake_img, (real_img.shape[1], real_img.shape[0]))

            psnr = calculate_psnr(real_img, fake_img)
            ssim_value = calculate_ssim(real_img, fake_img)
            ergas = calculate_ergas(real_img, fake_img)

            psnr_values.append(psnr)
            ssim_values.append(ssim_value)
            ergas_values.append(ergas)

            #print(f"Image: {real_file}, PSNR: {psnr}, SSIM: {ssim_value}")

    # Calculate average PSNR and SSIM
    avg_psnr = np.mean(psnr_values)
    avg_ssim = np.mean(ssim_values)
    avg_ergas = np.mean(ergas_values)

    print(f"Average PSNR: {avg_psnr}")
    print(f"Average SSIM: {avg_ssim}")
    print(f"Average ERGAS: {avg_ergas}")

if __name__ == "__main__":
    main()

Average PSNR: 27.977277697180163
Average SSIM: 0.6931677061402193
Average ERGAS: 11.411951733233531


## degraded with styleGAN

In [11]:
def main():
    real_dir = '/kaggle/input/baseline'
    fake_dir = '/kaggle/input/degraded-stylegan'

    real_files = os.listdir(real_dir)
    fake_files = os.listdir(fake_dir)
    
    real_files = sorted(os.listdir(real_dir))
    fake_files = sorted(os.listdir(fake_dir))
    #print(fake_files)
    #print(real_files)
    psnr_values = []
    ssim_values = []
    ergas_values = []

    for real_file, fake_file in zip(real_files, fake_files):
        if real_file.endswith('.jpeg') and fake_file.endswith('.jpeg'):
            real_img = cv2.imread(os.path.join(real_dir, real_file))
            fake_img = cv2.imread(os.path.join(fake_dir, fake_file))

            if real_img.shape != fake_img.shape:
                fake_img = cv2.resize(fake_img, (real_img.shape[1], real_img.shape[0]))

            psnr = calculate_psnr(real_img, fake_img)
            ssim_value = calculate_ssim(real_img, fake_img)
            ergas = calculate_ergas(real_img, fake_img)

            psnr_values.append(psnr)
            ssim_values.append(ssim_value)
            ergas_values.append(ergas)

            #print(f"Image: {real_file}, PSNR: {psnr}, SSIM: {ssim_value}")

    # Calculate average PSNR and SSIM
    avg_psnr = np.mean(psnr_values)
    avg_ssim = np.mean(ssim_values)
    avg_ergas = np.mean(ergas_values)

    print(f"Average PSNR: {avg_psnr}")
    print(f"Average SSIM: {avg_ssim}")
    print(f"Average ERGAS: {avg_ergas}")

if __name__ == "__main__":
    main()

Average PSNR: 27.944460174648604
Average SSIM: 0.7469163320157207
Average ERGAS: 8.538527836613557
